In [1]:
import pandas as pd 
import numpy as np
import tensorly as tl
import plotly.express as px
import matplotlib.pyplot as plt 
from tensorly.decomposition import CP,tucker, parafac
from datetime import timedelta, date

In [2]:
# df_FR = pd.read_excel('FINANCIAL_RATIOS.xlsx', index_col =[0], skiprows=[1,2])
# df_FR.drop(list(df_FR.filter(regex='TSCO')), axis=1, inplace = True)
# df_FR.drop(list(df_FR.filter(regex='TTWO')), axis=1, inplace = True)
# df_FR.drop(list(df_FR.filter(regex='AXTI')), axis=1, inplace = True)
# df_FR.drop(list(df_FR.filter(regex='DISH')), axis=1, inplace = True)
# df_FR.drop(list(df_FR.filter(regex='TTEC')), axis=1, inplace = True)
# df_FR.drop(list(df_FR.filter(regex='ELTK')), axis=1, inplace = True)
# df_FR.drop(list(df_FR.filter(regex='LOGI')), axis=1, inplace = True)

# df_FR.to_excel('FINANCIAL_RATIOS_f.xlsx')


# df_TI = pd.read_excel('TECHNICAL_INDICATORS.xlsx')
# df_TI.columns = df_TI.columns.astype(str)

# df_TI.drop(list(df_TI.filter(regex='TSCO')), axis=1, inplace = True)
# df_TI.drop(list(df_TI.filter(regex='TTWO')), axis=1, inplace = True)
# df_TI.drop(list(df_TI.filter(regex='AXTI')), axis=1, inplace = True)
# df_TI.drop(list(df_TI.filter(regex='DISH')), axis=1, inplace = True)
# df_TI.drop(list(df_TI.filter(regex='TTEC')), axis=1, inplace = True)
# df_TI.drop(list(df_TI.filter(regex='ELTK')), axis=1, inplace = True)
# df_TI.drop(list(df_TI.filter(regex='LOGI')), axis=1, inplace = True)






### Currently the features are PE, PS, PB, MCAP and PX_LAST need to further improve it with feature selection.

In [4]:
df_FR = pd.read_excel('FINANCIAL_RATIOS_f.xlsx', index_col =[0])

price = None


FR_dic = {}
FR_list = ['PE', 'PX_LAST', 'PS','PB','MCAP']
n = 0 

for i,name in enumerate(FR_list) : 
    FR_dic[name] = (df_FR.iloc[:,n:n+208]
                    .set_index(df_FR.index)
                    )

    if name not in ['PX_LAST', 'MCAP'] : 
        FR_dic[name] = (FR_dic[name] - FR_dic[name].mean()) / FR_dic[name].std()
        
    if name in ['PX_LAST'] :
        price = FR_dic['PX_LAST']
        FR_dic[name] = np.log(FR_dic[name])
        FR_dic[name] = FR_dic[name].diff()
    
    if name in ['MCAP'] : 
        FR_dic[name] = FR_dic[name] - FR_dic[name].shift(+1)
    
    FR_dic[name] = FR_dic[name][1:]
    n +=210


COLUMN_NAMES = FR_dic['PE'].columns

















In [5]:
print(price)

            AMAT US Equity.1  COKE US Equity.1  WDFC US Equity.1  \
2006-11-10             17.62             60.52             32.86   
2006-11-17             18.12             64.71             32.48   
2006-11-24             18.52             64.32             32.53   
2006-12-01             17.73             61.78             32.16   
2006-12-08             18.47             66.51             32.99   
...                      ...               ...               ...   
2022-04-15            113.36            492.70            189.24   
2022-04-22            112.80            474.80            191.71   
2022-04-29            110.35            441.50            183.98   
2022-05-06            112.50            483.65            182.03   
2022-05-13            111.86            530.80            185.70   

            AAPL US Equity.1  KLAC US Equity.1  SEIC US Equity.1  \
2006-11-10             2.969             48.55            28.655   
2006-11-17             3.066             51.69 

### We attempt to predict $AAPL 's cumulative log returns 8 weeks from now based on historical data which has a lookback period of 26 weeks.


In [6]:
for col in FR_dic['PX_LAST'] : 
    if col.startswith('AAPL') : 
        y_predict = FR_dic['PX_LAST'][col]

AAPL_returns = pd.DataFrame(y_predict).set_axis(['Log Returns'], axis = 1)
AAPL_returns['Cumulative Log Returns'] = AAPL_returns['Log Returns'].cumsum()



def get_eight_week_returns(log_returns, look_back_duration = 26) : 
    sdate = AAPL_returns.index.values[0]
    edate = AAPL_returns.index.values[-1]
    s = (log_returns
     .reset_index()
     .iloc[look_back_duration:]
    )
    eight_week_returns = s.rolling(8).sum()
    eight_week_returns.index = list(pd.date_range(sdate ,edate + pd.to_timedelta(2, unit='D') ,freq='w') - pd.to_timedelta(2, unit='D'))[:-look_back_duration]
    return eight_week_returns.dropna() 


def to_simple_return(cum_log_ret) : 
    return np.exp(cum_log_ret) - 1

eight_ret = get_eight_week_returns(AAPL_returns['Log Returns'])

Y_train = eight_ret
# AAPL_simple_returns = to_simple_return(AAPL_returns['Cumulative Log Returns'][-1] )


In [7]:
# df_TI = pd.read_excel('TECHNICAL_INDICATORS_f.xlsx', skiprows=[0])
# df_TI.columns = df_TI.columns.astype(str)


In [8]:
# TI_dic = {}
# TI_list = ['ATR', 'BB', 'MACD_DIFF','MOM','ROC', 'RSI']
# n = 1

# for _, TI in enumerate(TI_list) : 
#     filter_col = [col for col in df_TI if col.startswith(TI)]
#     print(df_TI[filter_col].shape)
#     TI_dic[TI] = (df_TI[filter_col]
#                  .set_axis(COLUMN_NAMES, axis=1)
#                  .set_index(df_TI['Dates'])
#                  .iloc[:-3]
#                  )

In [9]:
# # df_FR = pd.read_excel('FINANCIAL_RATIOS.xlsx', index_col =[0], skiprows=[1,2])
# df_PX = pd.read_excel('PRICE_DATA.xlsx', index_col =[0], skiprows=[1,2])


# df_PX.drop(list(df_PX.filter(regex='TTWO')), axis=1, inplace = True)
# df_PX.drop(list(df_PX.filter(regex='TSCO')), axis=1, inplace = True)
# df_PX.drop(list(df_PX.filter(regex='AXTI')), axis=1, inplace = True)
# df_PX.drop(list(df_PX.filter(regex='DISH')), axis=1, inplace = True)
# df_PX.drop(list(df_PX.filter(regex='TTEC')), axis=1, inplace = True)
# df_PX.drop(list(df_PX.filter(regex='ELTK')), axis=1, inplace = True)
# df_PX.drop(list(df_PX.filter(regex='LOGI')), axis=1, inplace = True)

# df_PX.to_excel('PRICE_DATA_f.xlsx')

In [10]:
df_PX = pd.read_excel('PRICE_DATA_f.xlsx', index_col =[0])
PX_dic = {}


PX_list = ['PX_HIGH', 'PX_LOW', 'PX_MID', 'PX_LAST', 'PX_VOL']


n = 0 

for i,name in enumerate(PX_list) : 
    PX_dic[name] = (df_PX.iloc[:,n:n+208]
                    .set_index(df_PX.index)
                    .iloc[:-2]
                    .iloc[1:]
                    )
    n +=208


    
    # FR_dic[name] = (FR_dic[name] - FR_dic[name].mean()) / FR_dic[name].std()

# COLUMN_NAMES = PX_dic['PE'].columns

In [11]:
# list_of_dfs = df.iloc[::5,:]  


feature_tensor = np.stack(FR_dic.values(), axis=2)#



# There is 776 datasets to train on
# We first apply the tucker decomposition and then vectorize the values.

list_of_features = []
for i in range(0,len(feature_tensor)-26-7) : 
    # print(len(feature_tensor)-26-7)
    list_of_features.append(feature_tensor[i:i+26, :,:])


# Tucker decompose the feature vectors
list_of_decomposed_features = []
for t_features in list_of_features :
    core, factors = (tucker(t_features, rank = [13,20,5]))
    list_of_decomposed_features.append(core)


# print(feature_tensor.shape)


# print(feature_tensor)
# feature_tensor[::32,:,:].shape

# core,factors = tucker(feature_tensor, rank = [809,50,5])

# print(core.shape)


C:\Users\zackx\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


### We build a RandomForestRegressor model 

Random Forests is a supervised machine learning algorithim that uses multiple decision trees in aggregate to help make more stable and accurate predictions.


In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV


X_Cols = [tensor.flatten() for tensor in list_of_decomposed_features ]
Y_Cols = eight_ret

X_train, X_test, Y_train, Y_test = train_test_split(X_Cols, Y_Cols)
 

# rand_frst_reg = RandomForestRegressor(n_estimators=)



Hyperparameter tuning using RandomizedSearchCV

In [22]:
n_estimators = [100, 500, 1000, 2000]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10,120,10)]

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]


random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap


}



rf = RandomForestRegressor()

# Using 5 fold cross validation
# Search across 100 different combintations and use all available cores

rf_random = RandomizedSearchCV(
    estimator = rf,
    param_distributions= random_grid,
    n_iter = 100, 
    cv = 5, 
    verbose= 2,
    random_state=35,
    n_jobs = - 1
                        )

rf_random.fit(X_train, Y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


### Model Evaluation : Accuracy 

In [ ]:
import matplotlib.pyplot as plt 
print('Random grid: ', random_grid, '\n')

print('Best Parameters: ', rf_random.best_params_, ' \n')

predictions = rf_random.predict(X_test)

errors  = abs(predictions - Y_test)

print('Average absolute error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / Y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')



# Plot of the predicted log_returns vs actual returns
plt.plot(Y_test)
plt.plot(predictions, 'o')
plt.show()

